# Linear Variational Principle
This notebook demonstrates the Linear Variational Method to the particle in a box of length $10$ atomic units 
with a delta function potential centered at $x=5$ atomic units.  

We will optimize
the trial wavefunction given by 
\begin{equation}
\Phi(x) = \sum_{i=1}^N c_i \psi_i(x)
\end{equation}
where the coefficients $c_i$ are real numbers
and $\psi_i(x)$ are the energy eigenfunctions of the particle in a box with no potential:
\begin{equation}
\psi_n(x) = \sqrt{\frac{2}{10} } {\rm sin}\left(\frac{n \pi x}{10} \right).
\end{equation}

We will seek to minimize the energy functional through the expansion coefficients, where the
energy functional can be written as
\begin{equation}
E[\Phi(x)] = \frac{\int_0^{\infty} \Phi^* (x) \: \hat{H} \: \Phi(x) dx }{\int_0^{\infty} \Phi^* (x) \: \Phi(x) dx }.
\end{equation}


The Hamiltonian operator in the box is given by 
\begin{equation}
\hat{H} = -\frac{\hbar^2}{2m} \frac{d^2}{dx^2} + \delta(x-5);
\end{equation}
in natural units, $\hbar$ and $m$ are equal to 1.

$E[\Phi(x)]$ can be expanded as
\begin{equation}
E[\Phi(x)] \sum_{i=1}^N \sum_{j=1}^N c_i c_j S_{ij} = \sum_{i=1}^N \sum_{j=1}^N c_i c_j H_{ij}
\end{equation}
where 
\begin{equation}
S_{ij} = \int_0^L \psi_i(x) \psi_j(x) dx = \delta_{ij}
\end{equation}
and
\begin{equation}
H_{ij} = \int_0^L \psi_i(x) \hat{H} \psi_j(x) dx. 
\end{equation}

We can work out a general expression for the integrals $H_{ij}$
\begin{equation}
H_{ij} = \frac{\hbar^2 \pi^2 j^2}{2 m L^2} \delta_{ij} + \frac{2}{L} {\rm sin}\left( \frac{i \pi}{2} \right) {\rm sin}\left( \frac{j\pi}{2} \right).
\end{equation}

This expression can be implemented as a python function that takes the indices $i$ and $j$ and returns the value of the integral $H_{ij}$. 

In [2]:
import numpy as np
### Function to return integrals involving Hamiltonian and basis functions
def H_ij(i, j):
    if i==j:
        ham_int = np.pi**2 * j**2/(2 * 10**2) + (2/10) * np.sin(i*np.pi/2) * np.sin(j*np.pi/2)
    else:
        ham_int = (2/10) * np.sin(i*np.pi/2) * np.sin(j*np.pi/2)
        

    return ham_int

Next we will create a numpy array called $H_{mat}$ that can be used to store the Hamiltonian matrix elements.  We can start by considering a trial wavefunction that is an expansion of the first 3 PIB energy eigenfunctions, so our Hamiltonian in this case should be a 3x3 numpy array.

In [3]:
dim = 3
H_mat = np.zeros((dim,dim))

You can use two nested $for$ loops along with your $H_{ij}$ function to fill out the values of this matrix.  Note that the indices for numpy arrays start from zero while the quantum numbers for our system start from 1, so we must offset our quantum numbers by +1 relative to our numpy array indices.

In [4]:
### loop over indices of the basis you are expanding in
### and compute and store the corresponding Hamiltonian matrix elements
for i in range(0,dim):
    for j in range(0,dim):
        H_mat[i][j] = H_ij(i+1, j+1)

### Print the resulting Hamiltonian matrix
print(H_mat)

[[ 2.49348022e-01  2.44929360e-17 -2.00000000e-01]
 [ 2.44929360e-17  1.97392088e-01 -2.44929360e-17]
 [-2.00000000e-01 -2.44929360e-17  6.44132198e-01]]


Before systematically identifying the optimal coefficients, it is instructive to try a few "trial" wavefunctions "by hand".  A few suggestions include:
\begin{equation}
{\bf c} = \left(1, 0, 0 \right) \;  {\bf c} = \left(0, 1, 0 \right) \: {\bf c} = \left(0, 0, 1 \right)
\end{equation}
\begin{equation}
{\bf c} = \left(\sqrt{1/2}, \sqrt{1/2}, 0 \right) \;  {\bf c} = \left(0, \sqrt{1/2}, \sqrt{1/2} \right) \: {\bf c} = \left(\sqrt{1/2}, 0, \sqrt{1/2} \right)
\end{equation}

In Matrix form, the energy functional can be computed as follows:
\begin{equation}
E = \frac{{\bf c}^t {\bf H} {\bf c}}{{\bf c}^t {\bf c}}
\end{equation}
where ${\bf c}^t$ is just the transpose of ${\bf c}$.  Using numpy, this can be done with the Hamiltonian matrix defined above and a vector ${\bf c} = \left(1, 0, 0 \right)$ as follows: 

In [7]:
### create an empty numpy array for the c vector
c = np.zeros(dim)
### assign c vector to be (1, 0, 0)
c[0] = 1

### compute H_mat * c and store it to a new array called Hc
Hc = np.dot(H_mat,c)

### compute c^t * Hc and store it to a variable E_expt_num
E_expt_num = np.dot(np.transpose(c),Hc)
### compute c^t * c and store it to a variable norm
norm = np.dot(np.transpose(c),c)
### print the result
print("The Energy Expectation Value is ",E_expt_num/norm,"Atomic Units")


The Energy Expectation Value is  0.2493480220054468 Atomic Units


Continue evaluating the energy of different trial wavefunctions by changing the values of the ${\bf c}$ vector and repeating the calculation above.  Does increasing the contribution of excited states impact the energy as you expect?  Explain.

Finally, to get the optimal values of the ${\bf c}$ vector, we can find the set of vectors (there will be 3) that satisfy the eigenvalue equation we wrote before.  We can use the $eig$ function of numpy to do this in one line:

In [8]:
### compute eigenvalues and eigenvectors of H_mat
### store eigenvalues to E_opt and eigenvectors to c_opt
E_opt, c_opt = np.linalg.eig(H_mat)

### print lowest eigenvalues corresponding to the 
### variational estimate of the ground state energy
print("Ground state energy with potential is approximately ",np.min(E_opt))
print("Ground state energy of PIB is ",np.pi**2/(200))




Ground state energy with potential is approximately  0.16573541893898724
Ground state energy of PIB is  0.04934802200544679


### Questions To Think About!
1.  Is the energy you calculated above higher or lower than the ground state energy of the ordinary particle in a box system (without the delta function potential)?
Answer:  The energy calculated to approximate the ground state energy of the PIB + Potential using the linear variational method is higher than the true PIB ground state energy (0.165 atomic units for the PIB + Potential compared to 0.0493 atomic units for the ordinary PIB).  The addition of the potential should increase the ground state energy because it is repulsive.
2.  Why do you think mixing in functions that correspond to excited states in the ordinary particle in a box system actually helped to improve (i.e. lower) your energy in the system with the delta function potential?
Answer:  Certain excited states (all states with even $n$) go to zero at the center of the box, and the repulsive potential is localized to the center of the box.  Therefore, all excited states with even $n$ will move electron density away from the repulsive potential, which can potentially loer the energy.
3.  Increase the number of basis functions to 6 (so that ${\bf H}$ is a 6x6 matrix and ${\bf c}$ is a vector with 6 entries) and repeat your calculation of the variational estimate of the ground state energy.  Does the energy improve (lower) compared to what it was when 3 basis functions were used?
Answer:  Yes, the energy improves.  With 3 basis functions, the ground state energy is approximated to be 0.165 atomic units and with 6 basis functions, the ground state energy is approximated to be 0.155 atomic units.  The added flexibility of these additional basis functions (specifically more basis functions with $n$ even) allows greater flexibility in optimizing a wavefunction that describes an electron effectively avoiding the repulsive potential in the center of the box.